## Convert data to AnnData-Zarr and OME-TIFF (alternative)

Instead of polygons (diamonds) for the pseudo-segmentations for spots on the Visium image, we will generate bitmask (circle) segmentations.

In [ ]:
import scanpy as sc
import numpy as np
import scipy.cluster
from scipy import sparse
from skimage.draw import disk
from vitessce.data_utils import (
    to_diamond,
    multiplex_img_to_ome_tiff,
    rgb_img_to_ome_tiff,
    optimize_adata,
    to_dense,
    VAR_CHUNK_SIZE,
    sort_var_axis,
)
from os.path import join

In [ ]:
output_img = join("processed_data", "alternate.image.ome.tiff")
output_segmentations = join("processed_data", "alternate.bitmask.ome.tiff")
output_adata = join("processed_data", "alternate.adata.zarr")

## Load data from ScanPy example dataset

ScanPy contains several [example datasets](https://scanpy.readthedocs.io/en/stable/api.html#module-scanpy.datasets), which can be loaded as AnnData objects. In this notebook, we load the [V1_Human_Lymph_Node](https://scanpy.readthedocs.io/en/stable/generated/scanpy.datasets.visium_sge.html#scanpy.datasets.visium_sge) 10x Genomics Visium example.

In [ ]:
adata = sc.datasets.visium_sge(sample_id="V1_Human_Lymph_Node", include_hires_tiff=True)

## Perform QC, filtering, normalization, dimensionality reduction, clustering

In [ ]:
# Reference: https://scanpy-tutorials.readthedocs.io/en/latest/spatial/basic-analysis.html
# Calculate QC metrics
adata.var_names_make_unique()
adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)

# Perform basic filtering
sc.pp.filter_cells(adata, min_counts=5000)
sc.pp.filter_cells(adata, max_counts=35000)
adata = adata[adata.obs["pct_counts_mt"] < 20]
sc.pp.filter_genes(adata, min_cells=10)

# Perform normalization
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)
# Determine the top 300 highly variable genes.
sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=300)

# Dimensionality reduction and clustering
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata, key_added="clusters")

## Reorder the gene axis

In [ ]:
# Hierarchical clustering of genes for optimal gene ordering
X_hvg_arr = adata[:, adata.var['highly_variable']].X.toarray()
X_hvg_index = adata[:, adata.var['highly_variable']].var.copy().index

Z = scipy.cluster.hierarchy.linkage(X_hvg_arr.T, method="average", optimal_ordering=True)

# Get the hierarchy-based ordering of genes.
num_cells = adata.obs.shape[0]
highly_var_index_ordering = scipy.cluster.hierarchy.leaves_list(Z)
highly_var_genes = X_hvg_index.values[highly_var_index_ordering].tolist()

all_genes = adata.var.index.values.tolist()
not_var_genes = adata.var.loc[~adata.var['highly_variable']].index.values.tolist()

def get_orig_index(gene_id):
    return all_genes.index(gene_id)

var_index_ordering = list(map(get_orig_index, highly_var_genes)) + list(map(get_orig_index, not_var_genes))

# Create a new *ordered* gene expression dataframe.
adata = adata[:, var_index_ordering].copy()
adata.obsm["X_hvg"] = adata[:, adata.var['highly_variable']].X.copy()

## Process the spatial data

### Save the image to OME-TIFF

In [ ]:
# Write img_arr to OME-Zarr.
# Need to convert images from interleaved to non-interleaved (color axis should be first).
img_hires = adata.uns['spatial']['V1_Human_Lymph_Node']['images']['hires']
img_arr = np.transpose(img_hires, (2, 0, 1))
# Convert values from [0, 1] to [0, 255].
img_arr *= 255.0

rgb_img_to_ome_tiff(img_arr, output_img, axes="CYX", img_name="H & E Image")

### Generate pseudo-segmentations

In [ ]:
# Unclear what the exact scale factor is required to align
# the spots to the image. Through trial and error / manual binary search
# of values I arrived at:
scale_factor = 1 / 5.87
adata.obsm['spatial'] = (adata.obsm['spatial'] * scale_factor)

In [ ]:
img_arr.shape # (c, y, x)

### Create a bitmask array

In [ ]:
bitmask_arr = np.zeros((img_arr.shape[2], img_arr.shape[1]), dtype=np.uint16)
bitmask_arr.shape # (x, y)

Use the [disk](https://scikit-image.org/docs/stable/api/skimage.morphology.html#skimage.morphology.disk) function to create a circle with a given radius at each spatial coordinate.

In [ ]:
radius = 10
for i in range(num_cells):
    x = adata.obsm['spatial'][i, 0]
    y = adata.obsm['spatial'][i, 1]
    bitmask_arr[disk((x, y), radius)] = i+1 # add one (0 is reserved for the background)

# Update the array axes so they are in CYX order to enable conversion to OME-TIFF.
bitmask_arr = bitmask_arr.transpose((1, 0)) # (y, x)
bitmask_arr = bitmask_arr[np.newaxis, :] # (c, y, x)
bitmask_arr.shape

### Save the bitmask to OME-TIFF

In [ ]:
multiplex_img_to_ome_tiff(bitmask_arr, ["Spots"], output_segmentations, axes="CYX")

## Save the AnnData object to AnnData-Zarr format

In [ ]:
# Store the current observation index in a new column called "spot_id"
adata.obs = adata.obs.reset_index().rename(columns={"index": "spot_id"})
# Create an integer index starting at 1 (0 is reserved for the background)
adata.obs.index = list(range(1, num_cells+1))
adata.obs

In [ ]:
adata = optimize_adata(
    adata,
    obs_cols=["spot_id", "clusters"],
    var_cols=["highly_variable"],
    obsm_keys=["X_hvg", "spatial", "X_umap", "X_pca"],
    optimize_X=True,
    # Vitessce plays nicely with dense matrices saved with chunking
    # and this one is small enough that dense is not a huge overhead.
    to_dense_X=True,
)
adata.write_zarr(output_adata, chunks=[adata.shape[0], VAR_CHUNK_SIZE])